In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm
from collections import defaultdict

In [2]:
threshold = 1e-3

def compute_token_sparsity(token_activations, threshold=threshold):
    """Sparsity across tokens (spatial dimension), given [B, N, C]"""
    sparsity_dict = {}
    for name, act in token_activations.items():
        if act is None or act.numel() == 0:
            continue
        sparse_ratio = (act.abs() < threshold).float().mean().item()
        sparsity_dict[name] = sparse_ratio
    return sparsity_dict

def compute_feature_sparsity(channel_activations, threshold=threshold):
    """Sparsity across feature dimensions, using [B, C]"""
    sparsity_dict = {}
    for name, vec in channel_activations.items():
        if vec is None or vec.numel() == 0:
            continue
        sparse_ratio = (vec.abs() < threshold).float().mean().item()
        sparsity_dict[name] = sparse_ratio
    return sparsity_dict


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
dataset_path = "../aircraft_dataset"
batch_size = 16

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_path = os.path.join(dataset_path, f"test")
test_ds = ImageFolder(root=test_path, transform=transform)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

num_classes = len(test_ds.classes)

print(num_classes, len(test_ds))

70 1000


In [5]:
from instrumented_models.swin_instrumented import SwinTiny_InternalRepresentation

model = SwinTiny_InternalRepresentation(num_classes=num_classes, pretrained=False)

weights = torch.load("swin_epoch_25.pth", map_location=device)
weights = {"model." + k: v for k, v in weights.items()}
model.load_state_dict(weights)

model.to(device)
model.eval()

print(model.__class__.__name__)

num_params = sum(p.numel() for p in model.parameters())
print(f"model parameters: {num_params}")

d:\conda_envs\fyp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SwinTiny_InternalRepresentation
model parameters: 27573184


In [6]:
token_sparsity_acc = defaultdict(list)
feature_sparsity_acc = defaultdict(list)

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)

        model(images)

        token_acts = model.get_token_activations()
        channel_acts = model.get_channel_activations()

        # Sparsity computation
        token_sparse = compute_token_sparsity(token_acts)
        feat_sparse = compute_feature_sparsity(channel_acts)

        for name, val in token_sparse.items():
            token_sparsity_acc[name].append(val)
        for name, val in feat_sparse.items():
            feature_sparsity_acc[name].append(val)

# Averaging results
token_sparsity = {name: sum(vals) / len(vals) for name, vals in token_sparsity_acc.items()}
feature_sparsity = {name: sum(vals) / len(vals) for name, vals in feature_sparsity_acc.items()}

print("\nToken (Spatial) Sparsity:")
for k, v in token_sparsity.items():
    print(f"{k}:\t{v:.4f}")

print("\nFeature (Channel-like) Sparsity:")
for k, v in feature_sparsity.items():
    print(f"{k}:\t{v:.4f}")


100%|██████████| 63/63 [00:49<00:00,  1.26it/s]


Token (Spatial) Sparsity:
stage_0_block_0:	0.0028
stage_0_block_1:	0.0036
stage_1_block_0:	0.0026
stage_1_block_1:	0.0026
stage_2_block_0:	0.0031
stage_2_block_1:	0.0028
stage_2_block_2:	0.0021
stage_2_block_3:	0.0020
stage_2_block_4:	0.0019
stage_2_block_5:	0.0022
stage_3_block_0:	0.0019
stage_3_block_1:	0.0004

Feature (Channel-like) Sparsity:
stage_0_block_0:	0.0049
stage_0_block_1:	0.0060
stage_1_block_0:	0.0046
stage_1_block_1:	0.0044
stage_2_block_0:	0.0050
stage_2_block_1:	0.0048
stage_2_block_2:	0.0035
stage_2_block_3:	0.0031
stage_2_block_4:	0.0027
stage_2_block_5:	0.0029
stage_3_block_0:	0.0022
stage_3_block_1:	0.0005
